In [25]:
### to-do ###
# compare rates with electronic police report data where rtcc footage is not requested 

In [26]:
import pandas as pd

def clean():
    df1 = pd.read_csv("electronic_police_report_2018.csv")
    df2 = pd.read_csv("electronic_police_report_2019.csv")
    df3 = pd.read_csv("electronic_police_report_2020.csv")
    df4 = pd.read_csv("electronic_police_report_2021.csv")
    df5 = pd.read_csv("electronic_police_report_2022.csv")

    dfs = [df1, df2, df3, df4, df5]
    dfa = pd.concat(dfs, join="outer")
    
    dfb = pd.read_csv("rtcc.csv", encoding="cp1252").rename(columns={"Item_number": "item_number"})

    df = pd.merge(dfa, dfb, on="item_number")
    return df

In [27]:
df = clean

In [28]:
def drop_rows_missing_race(df):
    df.loc[:, "offender_race"] = df.offender_race.fillna("").str.lower().str.strip()\
    .str.replace("unknown", "", regex=False)
    # .str.replace("hispanic", "non-black", regex=False)\
    # .str.replace("asian", "non-black", regex=False)\
    # .str.replace("white", "non-black", regex=False)\
    # .str.replace(r"amer\. ind\.", "non-black", regex=True)
    return df[~((df.offender_race == ""))]


def filter_gender(df):
    df.loc[:, "offender_gender"] = (
        df.offender_gender.str.lower().str.strip().fillna("").str.replace("unknown", "", regex=False)
    )
    df = df[df.offender_gender.isin(["male"])]
    return df[~((df.offender_gender == ""))]


def filter_arrested(df):
    df.loc[:, "offenderstatus"] = df.offenderstatus.str.lower().str.strip().fillna("")
    df = df[df.offenderstatus.isin(["arrested"])]
    return df[~((df.offenderstatus == ""))]


def filter_year(df):
    df = df[df.year.astype(str).isin(["2018", "2019", "2020", "2021", "2022"])]
    # the following figures are percentages of the "offender_race" value
    ### 2018 - 2022
    """
      black         0.768292
      unknown       0.152651
      white         0.068554
      hispanic      0.009226
      asian         0.001065
      -amer. ind.    0.000213
    """
    ### 2022
    """
      black      0.660252
      unknown    0.297557
      white      0.039230
      asian      0.002961
    """
    ### 2021
    """
      black       0.663802
      unknown     0.279270
      white       0.055209
      asian       0.001074
      hispanic    0.000644
    """
    ### 2020
    """
      black         0.867308
      unknown       0.069408
      white         0.046369
      hispanic      0.016040
      amer. ind.    0.000875
    """
    ### 2019
    """
      black       0.849009
      white       0.091532
      unknown     0.044685
      hispanic    0.013694
      asian       0.001081
    """
    ### 2018
    """
      black       0.804891
      white       0.129187
      unknown     0.046252
      hispanic    0.018075
      asian       0.001595
    """
    return df


def extract_years(df):
    years = df.occurred_date_time.astype(str).str.extract(r"(\w{4})")

    df.loc[:, "year"] = years[0]
    return df

In [29]:
def filter_data():
  df = clean()
  df = df.pipe(filter_year).pipe(drop_rows_missing_race).pipe(filter_gender)
  return df

In [30]:
df = filter_data()

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
df["rtcc_footage_request_tally"] = 1

In [32]:
df = df[["offender_race", "rtcc_footage_request_tally", "offender_gender"]]
df = df.groupby(["offender_race", "offender_gender"]).rtcc_footage_request_tally.count()
df = pd.DataFrame(df).reset_index()

In [33]:
df.loc[(df.offender_race == "black"), "population"] = "105498"
df.loc[(df.offender_race == "white"), "population"] = "65756"
df.loc[(df.offender_race == "hispanic"), "population"] = "11205"
df.loc[(df.offender_race == "amer. ind."), "population"] = "360"
df.loc[(df.offender_race == "asian"), "population"] = "5499"

In [34]:
def calc_rate(dfb_total, population=""):
    population = int(population)
    rate = (( dfb_total / population) * 1000)
    return rate

In [35]:
df["rate_per_1000_persons"] = df.apply(lambda x: calc_rate(x.rtcc_footage_request_tally, x.population), axis=1)

In [36]:
df.sort_values("offender_race")

,offender_race,offender_gender,rtcc_footage_request_tally,population,rate_per_1000_persons
0,amer. ind.,male,3,360,8.333333
1,asian,male,15,5499,2.727769
2,black,male,9550,105498,90.523043
3,hispanic,male,108,11205,9.638554
4,white,male,781,65756,11.877243
